### **_PART\_1: Import Packages and Connect to Database_**

In [ ]:
# CREATED BY KEMI ADEJUMO
#IMPORT ALL PACKAGES 
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from openpyxl import load_workbook
import tkinter as tk
from tkinter import ttk
import pyodbc as odbc
import pandas as pd
import numpy as np
import glob
import ctypes
import warnings
import datetime
warnings.filterwarnings("ignore")

DRIVER_NAME = 'SQL_SERVER'
SERVER_NAME = 'SERVER_NAME'
DATABASE_NAME = 'DATABASE_NAME'
UID= <username>
PWD= <password>
connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE_NAME={DATABASE_NAME};
    Trust_Connection=yes;""" 

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect":connection_string})
engine = create_engine(connection_url, module=odbc)

### _**PART\_2: OLd POGList Products**_     _(Yo_<span style="color: var(--vscode-foreground); font-size: 14px;"><em>u will need POGs/SSN, ResetDate, and Stores)</em></span>

In [ ]:
#Old POG List Products
sql_statement= f"""
With Start as (
    
SELECT  DBVersionKey,  MAX(DBDateEffectiveFrom) as 'Live Date'

FROM DATABASE_NAME.Planogram

WHERE Status1 IN ('Live', 'Pending', 'Merchandising Complete')
AND SSN IN('D060_SSNAME',
'D060_SSNAME',
'D060_SSNAME',
'D060_SSNAME',
'D060_SSNAME',

)


AND (DBDateEffectiveFrom <= '2023-08-21 00:00:00.000')

GROUP BY DBVersionKey

)

SELECT fp.STORE#, Start.DBVersionKey, Start.[Live Date], pg.DBKey, pg.DBStatus ,pr.Name, pg.Desc37,pr.UPC

FROM Start JOIN DATABASE_NAME.Planogram as pg ON Start.DBVersionKey = pg.DBKey

JOIN  DATABASE_NAME.Section as sc on pg.DBVersionKey = sc.DBParentPlanogramKey

JOIN  DATABASE_NAME.floorplan as fp on sc.DBParentFloorplanKey = fp.DBKey

JOIN  DATABASE_NAME.position AS ps ON pg.DBKey = ps.DBParentPlanogramKey 

JOIN  DATABASE_NAME.product as pr on ps.DBParentProductKey = pr.DBKey

WHERE fp.STORE IN ('DIGITS','DIGITS');

"""

old_list = pd.read_sql_query(sql_statement,engine)
old_list

**_PART\_3: New POG List Products_**

In [ ]:
#New POG List Products

sql_statement_2 = f"""
With Second as (

SELECT  DBVersionKey,  MAX(DBDateEffectiveFrom) as 'Live Date'

FROM DATABASE_NAME.Planogram

WHERE Status1 IN ('Live', 'Pending', 'Mechandising Complete')
AND SSN IN('D060_SSNAMEX','D060_SSNAMEZ')

-- Reset date + 14 days
AND (DBDateEffectiveFrom <= '2023-09-03 00:00:00.000')

GROUP BY DBVersionKey
)


---

SELECT Second.DBVersionKey, Second.[Live Date], pg.DBKey, pg.DBStatus ,pr.Name, pg.Desc37,pr.UPC

FROM Second
JOIN DATABASE_NAME.Planogram as pg ON Second.DBVersionKey = pg.DBversionKey

--Now, connect to product table to pull products. 

JOIN  DATABASE_NAME.position AS ps ON pg.DBKey = ps.DBParentPlanogramKey 
JOIN  DATABASE_NAME.product as pr on ps.DBParentProductKey = pr.DBKey


WHERE pg.DBKey IN ('DIGITS','DIGITS')


"""
new_list = pd.read_sql_query(sql_statement_2,engine)
new_list


**PART\_4: Comparing OLd And New UPC list; Old items that are not in new**

In [ ]:
#This will compare the old and new list product: old list upc that are not in new list will be printed (delete_upc)

def compare_dataframes(old_list, new_list):
    # Find unique UPCs in DataFrame A that are not in DataFrame B
    unique_upcs = old_list[~old_list['UPC'].isin(new_list['UPC'])]

    if unique_upcs.empty:
        print("No unique UPCs found in DataFrame A.")
    else:
        print("Unique UPCs in DataFrame A:")
        return unique_upcs

delete_upc= compare_dataframes(old_list, new_list)

delete_upc

**PART\_5: Find Other POGs attach to the delete UPCs**

In [ ]:
#Convert the UPC column to tuples, and adding this format '000000123','00000000023

#withzeros = delete_upc['UPC'].apply(lambda x: f'000{x}')

withzerosx =tuple(delete_upc['UPC'].unique())

delete_upcs = withzerosx








In [ ]:
# Take the delete upcs and find any other pogs attached to it.

sql_statement1= f"""
SELECT DISTINCT pr.UPC, 
pg.Desc37 as SSN,
pr.Name AS Mask_Product_Name,
pr.Desc32 AS Comm,
pr.Size,
pr.UOM,
pg.DBKey,
pg.DBVersionKey,
fp.Desc22

FROM DATABASE_NAME.Planogram as pg

JOIN  DATABASE_NAME.Section as sc on pg.DBVersionKey = sc.DBParentPlanogramKey

JOIN  DATABASE_NAME.floorplan as fp on sc.DBParentFloorplanKey = fp.DBKey

JOIN  DATABASE_NAME.position AS ps ON pg.DBKey = ps.DBParentPlanogramKey 

JOIN  DATABASE_NAME.product as pr on ps.DBParentProductKey = pr.DBKey

WHERE 
pr.UPC IN {delete_upcs} 

AND (pg.Status1 = 'Live' OR pg.Status1 = 'Pending' 
OR  pg.Status1 = 'Merchandising Complete') 
AND fp.STORE# IN ('DIGITS','DIGITS')
;
"""
extend = pd.read_sql_query(sql_statement1,engine)
#.drop_duplicates()
extend
#products.UPC IN {UPCs} AND


**PART\_6: Compare Part 4 and 5: if part 5 in 4 (all) Proceed with Part 4 i.e keep,** if not then some upc have other POGs Proceed with Part 7

In [ ]:
def find_extends(delete_upcs, extend):
    diff = extend[~extend['SSN'].isin(delete_upc['Desc37'])]

    if diff.empty:
        print("No extend POGs found in DataFrame A.")
    else:
        print("Extend POGs in DataFrame A:")
        return diff

extend_upc = find_extends(delete_upcs, extend)

extend_upc

**PART\_7 : This Step is contingent to part 6**

Take part 6 delete the UPCs from PArt 4 and Keep ther rest of Part 4(this will the final products that will be submitted for deletes)

In [ ]:
extend_upc_tuple = tuple(extend_upc['UPC'].unique())

# Function to delete rows with UPC values present in the 'delete_upc[UPC]' column
def delete_rows_with_tuple(delete_upc_df, extend_upc_tuple):
    # Use 'isin' to check if each UPC value in 'delete_upc_df' exists in the 'extend_upc_tuple'
    mask = delete_upc_df['UPC'].isin(extend_upc_tuple)
    
    # Drop rows where the UPC value is present in 'extend_upc_tuple'
    delete_upc_df = delete_upc_df[~mask]
    
    return delete_upc_df

# Call the function to delete rows with UPC values present in 'extend_upc_tuple'
call_func = delete_rows_with_tuple(delete_upc, extend_upc_tuple)

# Print the remaining 'delete_upc[UPC]' after deleting the rows with UPC values in 'extend_upc_tuple'

delete_upc_final = pd.DataFrame(call_func)

delete_upc_final


